# Visualización dinámica del histórico de ventas por producto + Predicción

Este notebook permite:

- Cargar el archivo `sell-in.txt` y `forecast_product_phase1_con_real.csv`,
- Ver el histórico de toneladas (`tn`) por producto y período,
- Superponer en el gráfico los valores **predichos** por modelo (`tn_predicho`).


In [17]:

import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact


## Carga de archivos y join con predicción

In [20]:
# Cargar archivo de ventas
df_sellin = pd.read_csv("C:\\Developer\\Laboratorio_III\\data\\sell-in.txt", sep='\t')
df_sellin['periodo'] = df_sellin['periodo'].astype(str)

# Agregar toneladas por período y producto (histórico real)
df_hist = df_sellin.groupby(['periodo', 'product_id'])['tn'].sum().reset_index()

# Cargar predicciones con real incluido
df_forecast = pd.read_csv("C:\\Developer\\Laboratorio_III\\notebooks\\forecast_product_phase1_con_real.csv")
df_forecast = df_forecast.rename(columns={'tn': 'tn_predicho'})

# Merge SOLO por product_id (tn_predicho es fijo por producto)
df_merged = df_hist.merge(df_forecast[['product_id', 'tn_predicho']], on='product_id', how='left')



## Visualización interactiva de ventas reales y predicción

In [23]:
@interact(producto_id=sorted(df_merged['product_id'].unique()))
def plot_producto_con_pred(producto_id):
    datos = df_merged[df_merged['product_id'] == producto_id].sort_values('periodo')
    pred = datos['tn_predicho'].iloc[0]

    plt.figure(figsize=(10, 4))
    plt.plot(datos['periodo'], datos['tn'], marker='o', color='steelblue', label='Ventas reales')

    if not pd.isna(pred):
        plt.axhline(y=pred, color='darkorange', linestyle='--', label=f'Predicción ARIMA: {pred:.2f}')

    plt.title(f"Evolución histórica de ventas - Producto {producto_id}")
    plt.xlabel("Periodo")
    plt.ylabel("Toneladas")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()




interactive(children=(Dropdown(description='producto_id', options=(20001, 20002, 20003, 20004, 20005, 20006, 2…

In [ ]:
@interact(producto_id=sorted(df_merged['product_id'].unique()))
def plot_producto_con_pred(producto_id):
    # Filtrar la serie completa del producto
    datos = df_merged[df_merged['product_id'] == producto_id].sort_values('periodo')

    # Extraer el valor predicho y confirmar si existe
    pred_row = datos[datos['periodo'] == '201912']
    valor_predicho = pred_row['tn_predicho'].iloc[0] if not pred_row.empty else None

    plt.figure(figsize=(10, 4))

    # Línea con la evolución histórica de ventas
    plt.plot(datos['periodo'], datos['tn'], marker='o', color='steelblue', label='Ventas reales')

    # Agregar punto rojo en 201912 si existe predicción
    if valor_predicho is not None and not pd.isna(valor_predicho):
        plt.scatter('201912', valor_predicho, color='red', s=100, zorder=5, label=f'Predicción ARIMA (201912): {valor_predicho:.2f}')

    plt.title(f"Evolución de ventas - Producto {producto_id}")
    plt.xlabel("Periodo")
    plt.ylabel("Toneladas")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


interactive(children=(Dropdown(description='producto_id', options=(20001, 20002, 20003, 20004, 20005, 20006, 2…